In [1]:
!pip install librosa==0.7.0
!pip install musdb

    100% |████████████████████████████████| 1.6MB 18.5MB/s ta 0:00:01
    100% |████████████████████████████████| 296kB 38.1MB/s ta 0:00:01
    100% |████████████████████████████████| 327kB 40.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for librosa ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/49/1d/38/c8ad12fcad67569d8e730c3275be5e581bd589558484a0f881
  Running setup.py bdist_wheel for audioread ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/b9/64/09/0b6417df9d8ba8bc61a7d2553c5cebd714ec169644c88fc012
  Running setup.py bdist_wheel for resampy ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/fa/c1/56/e0e12c6f7f3d2cdea9712b35136a2d40a7817c6210ec096485
Successfully built librosa audioread resampy
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 512kB 34.8MB/s ta 0:00:01
    100% |███████████████████

In [2]:
%matplotlib inline 
import numpy as np
import librosa
import librosa.display
import musdb
import os, os.path
import sys
import matplotlib.pyplot as plt
from PIL import Image
import scipy.io.wavfile

In [6]:
path = '/dltraining/dltraining/datasets/musdb18/'
save_path = '/dltraining/dltraining/datasets/mel_spec_db/'
duration = 11.87
hop_length = 1024
n_fft = 4094
n_mels = 512
sample_rate = 44100
samples_per = int(sample_rate * duration)

mus_train = musdb.DB(root=path, subsets="test")
  
for i, track in enumerate(mus_train):
    sys.stdout.write("\rSlicing Song {}...".format(i + 1))
    sys.stdout.flush()    
    
    mix_clip = track.audio
    #accomp_clip = track.targets['accompaniment'].audio
    vocal_clip = track.targets['vocals'].audio
    
    total_samples = mix_clip.shape[0]
    num_splits = int(total_samples / (duration*sample_rate))
    
    for j in range(num_splits):
        mix_section = mix_clip[(samples_per * j):(samples_per * (j + 1)), :]
        #accomp_section = accomp_clip[(samples_per * j):(samples_per * (j + 1)), :]
        vocal_section = vocal_clip[(samples_per * j):(samples_per * (j + 1)), :]
    
        stft = librosa.stft(mix_section[:, 0], n_fft=n_fft, hop_length=hop_length)
        stft_mag, stft_phase = librosa.magphase(stft)
        mix_stft_db = librosa.amplitude_to_db(stft_mag)
    
        stft = librosa.stft(vocal_section[:, 0], n_fft=n_fft, hop_length=hop_length)
        stft_mag, stft_phase = librosa.magphase(stft)
        vocal_stft_db = librosa.amplitude_to_db(stft_mag)   
    
        mix_mel = librosa.feature.melspectrogram(mix_section[:, 0], n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, sr=sample_rate)
        mix_mel_db = librosa.power_to_db(mix_mel)
    
        vocal_mel = librosa.feature.melspectrogram(vocal_section[:, 0], n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, sr=sample_rate)
        vocal_mel_db = librosa.power_to_db(vocal_mel)   
  
        save_name = save_path + 'Spectrogram/Mix/' + track.name + '_' + str(j)
        np.save(save_name, mix_stft_db)
    
        save_name = save_path + 'Spectrogram/Vocal/' + track.name + '_' + str(j)
        np.save(save_name, vocal_stft_db)
    
        save_name = save_path + 'Mel-Spectrogram/Mix/' + track.name + '_' + str(j)
        np.save(save_name, mix_mel_db)
    
        save_name = save_path + 'Mel-Spectrogram/Vocal/' + track.name + '_' + str(j)
        np.save(save_name, vocal_mel_db)
    

Slicing Song 50...

In [7]:
DIR = '/dltraining/dltraining/datasets/mel_spec_db/Spectrogram/Mix'
print(2 * len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

5318
